In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import joblib

# Load dataset
df = pd.read_csv("../features/audio_features_mfcc.csv")

# Split features and labels
X = df.drop(columns=['label', 'filename']).values  # Exclude label & filename
y = df['label'].values

# Load scaler and apply
scaler = joblib.load("scaler.pkl")
X = scaler.transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
input_shape = (X_train.shape[1],)
num_labels = 2

# Build model
model = Sequential([
    Dense(128, input_shape=input_shape, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_labels, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

# Save model
model.save("../models/audio_classification_model.h5")

# Evaluate model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")


Epoch 1/30


C:\Users\Pro 14\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Pro 14\.pyenv\pyenv-win\versions\3.11.8\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


270/270 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5375 - loss: 0.7353 - val_accuracy: 0.8028 - val_loss: 0.4465
Epoch 2/30
270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7943 - loss: 0.4613 - val_accuracy: 0.8958 - val_loss: 0.2813
Epoch 3/30
270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8563 - loss: 0.3490 - val_accuracy: 0.9069 - val_loss: 0.2303
Epoch 4/30
270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8787 - loss: 0.3057 - val_accuracy: 0.9088 - val_loss: 0.2147
Epoch 5/30
270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8772 - loss: 0.3096 - val_accuracy: 0.9370 - val_loss: 0.1611
Epoch 6/30
270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8938 - loss: 0.2703 - val_accuracy: 0.9352 - val_loss: 0.1683
Epoch 7/30
270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8980 - loss: 0.2634 - val_accuracy: 0.9398 - val_loss: 0.1503
Epoch 8/30
270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9070 - loss: 0.2386 - val_accuracy: 0.9431 - val_

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9729 - loss: 0.0824
Test Accuracy: 0.97


In [ ]:
import librosa
import numpy as np
import tensorflow as tf
import joblib

def extract_features(file_path, sample_rate=16000, n_mfcc=40):
    try:
        audio, sr = librosa.load(file_path, sr=sample_rate)
        mfccs_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
        return mfccs_features_scaled
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def predict_audio(file_path, model_path="../models/audio_classification_model.h5", scaler_path="scaler.pkl"):
    # Load model and scaler
    model = tf.keras.models.load_model(model_path)
    scaler = joblib.load(scaler_path)
    
    # Extract features
    features = extract_features(file_path)
    if features is None:
        return "Error: Could not process the audio file"
    
    # Standardize features
    features = scaler.transform([features])
    
    # Predict
    prediction = model.predict(features)
    predicted_label = np.argmax(prediction)
    confidence = prediction[0][predicted_label] * 100
    
    label_map = {0: "Real", 1: "Fake"}
    return f"Prediction: {label_map[predicted_label]} (Confidence: {confidence:.2f}%)"

# Example usage
file_path = "../audio_files/p_35769890_526.wav"  # Change to the path of your audio file
print(predict_audio(file_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Prediction: Fake (Confidence: 100.00%)


C:\Users\Pro 14\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
